# 2024C2: Continued Processing SST1 RSoXS Data
Use custom PyhyperScattering conda environment

## Imports

In [ ]:
# Imports
import PyHyperScattering as phs
import pathlib
import sys
import ast
import json
import datetime
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from tqdm.auto import tqdm
import dask.array as da
import hvplot.xarray
# from tiled.client import from_profile, from_uri

sys.path.append('/nsls2/users/alevin/local_lib')
from andrew_rsoxs_fxns import *

print(f'Using PyHyperScattering Version: {phs.__version__}')

# Set colormap
cmap = plt.cm.turbo.copy()
cmap.set_bad(cmap.get_under())

## Define paths & short functions

In [ ]:
# Define directory paths
userPath = pathlib.Path('/nsls2/users/alevin')
propPath = pathlib.Path('/nsls2/data/sst/proposals/2024-2/pass-313412')

outPath = propPath.joinpath('processed_data')
# jsonPath = outPath.joinpath('local_config')
maskPath = outPath.joinpath('masks')
zarrsPath = outPath.joinpath('rsoxs_zarrs')

In [ ]:
[f.name for f in zarrsPath.glob('*')]

## Load data from saved zarrs

In [ ]:
plan_of_interest = 'raw_rsoxs_carbon'
# plan_of_interest = 'raw_timestamped_rsoxs_1180'
loaded_DS = xr.open_zarr(zarrsPath.joinpath(f'cartesian_{plan_of_interest}_v1.zarr'))  #.compute()
loaded_DS = loaded_DS.where(loaded_DS>0)

# Compute any dask coordiantes
for coord_name, coord_data in loaded_DS.coords.items():
    if isinstance(coord_data.data, da.Array):
        loaded_DS.coords[coord_name] = coord_data.compute()
        
# Assign beam centers if known
loaded_DS['raw_intensity'].attrs['beamcenter_y'] = 508  # Good for 2024C2
loaded_DS['raw_intensity'].attrs['beamcenter_x'] = 477  # Good for 2024C2

display(loaded_DS)

# start_dict = ast.literal_eval(DA.start)

In [ ]:
# # Used for rsoxs 1180 eV

# for sample_name in loaded_DS.sample_name.values:
#     DA = loaded_DS['raw_intensity'].sel(sample_name=sample_name, polarization=90).squeeze()
#     cmin = float(DA.compute().quantile(0.0001))
#     cmax = float(DA.compute().quantile(0.995))
#     ax = DA.plot.imshow(norm=plt.Normalize(cmin, cmax), cmap=cmap, x='qx', y='qy')
#     ax.axes.set(title=f'{str(DA.sample_name.values)}, Energy = {int(DA.energy.values)} eV')
#     plt.show()
#     plt.close('all')

## Draw/check data & beamcenters & data

In [ ]:
# Set colormap
cmap = plt.cm.turbo.copy()
cmap.set_bad(cmap.get_under())

# # Choose a sample dataarray:
# bare_sin_DA = DS.sel(sample_name='BareSiN_1mm')
# print(DS.sample_name.values)
# sample_name = 'Y6_CB_2500'

### 1. Check raw images at a selected energy for all loaded scan configurations:

In [ ]:
loaded_DS

In [ ]:
# # sample_name = 'PM6-Y6_p5CN-2CF-3CB'
# sample_name = 'PM6_5CN-CF'
# # sample_DA = DS['raw_intensity'].sel(sample_name=sample_name)
# sample_DA = loaded_DS['raw_intensity'].sel(sample_name=sample_name)

# # energies = [270, 280, 282, 283, 284, 285, 286, 290]
# energies = np.round(np.linspace(280, 290, 8), 1)  # carbon
# # energies = np.round(np.linspace(380, 440, 8), 1)  # nitrogen

# pol = 0
# # pol = 90

# fg = sample_DA.sel(polarization=pol, method='nearest').sel(energy=energies, method='nearest').sel(
#             pix_x=slice(160, 780), pix_y=slice(240, 800)).plot.imshow(figsize=(18, 6),
#                 col='energy', col_wrap=4, norm=LogNorm(3e1, 1e4), cmap=cmap, x='qx', y='qy')
# fg.cbar.set_label('Intensity [arb. units]', rotation=270, labelpad=15)
# fg.fig.suptitle(f'{str(sample_DA.sample_name.data)},  Polarization = {pol}°', y=1.02)
# for axes in fg.axs.flatten():
#     axes.set(aspect='equal')

# plt.show()

In [ ]:
# # sample_name = 'PM6-Y6_p5CN-2CF-3CB'
# sample_name = 'PM6-Y6BO_p5CN-CB_rot'
# corr_sample_DA = loaded_DS['corr_intensity'].sel(sample_name=sample_name)

# # energies = [270, 280, 282, 283, 284, 285, 286, 290]
# energies = np.round(np.linspace(280, 290, 8), 1)  # carbon
# # energies = np.round(np.linspace(380, 440, 8), 1)  # nitrogen

# pol = 45

# fg = corr_sample_DA.sel(polarization=pol, method='nearest').sel(energy=energies, method='nearest').sel(
#             pix_x=slice(160, 780), pix_y=slice(240, 800)).plot.imshow(figsize=(18, 6), x='qx', y='qy',
#                 col='energy', col_wrap=4, norm=LogNorm(5e8, 1e11), cmap=cmap)
# fg.cbar.set_label('Intensity [arb. units]', rotation=270, labelpad=15)
# fg.fig.suptitle(f'{str(sample_name)},  Polarization = {pol}°', y=1.02)
# for axes in fg.axs.flatten():
#     axes.set(aspect='equal')

# plt.show()

### 2. Draw masks

In [ ]:
# loaded_DS.isel(datetime=0)

In [ ]:
# Define example image for mask & initialize phs DrawMask object:
# sample_name = 'PM6_5CN-CF'
# sample_DA = loaded_DS['raw_intensity'].sel(sample_name=sample_name)
sample_DA = loaded_DS['raw_intensity'].isel(datetime=0)

In [ ]:
%matplotlib widget

In [ ]:
## WAXS mask:
# waxs_mask_img = sample_DA.sel(polarization=0, energy=270, method='nearest').compute()
waxs_mask_img = sample_DA.compute()
# draw = phs.IntegrationUtils.DrawMask(waxs_mask_img, clim=(30, 1e3))
# draw = phs.IntegrationUtils.DrawMask(waxs_mask_img)
# draw.ui()

In [ ]:
# ## Save and load saxs drawn mask
# draw.save(maskPath.joinpath('2023C3_full_length_masks.json'))

### 3. Check beamcenters

In [ ]:
# # Define example image for mask & initialize phs DrawMask object:
# sample_name = 'PM6_5CN-CF'
# sample_DA = loaded_DS['raw_intensity'].sel(sample_name=sample_name)
# # sample_DA = sample_DA.where(sample_DA>0)

In [ ]:
waxs_mask_img = loaded_DS['raw_intensity'].sel(sample_name='PM6-Y6_CF', polarization=90).sel(energy=285, method='nearest')
waxs_mask_img

In [ ]:
plt.close('all')
# sample_DA.attrs['beamcenter_x'] = 450
# sample_DA.attrs['beamcenter_y'] = 510

# energy = 250
# energy = 400
# energy = 532
pix_x_slice = slice(None,None)
pix_y_slice = slice(None,None)
# pix_x_slice = slice(350,600)
# pix_y_slice = slice(350,650)

# waxs_mask_img = sample_DA.sel(polarization=0, energy=energy, method='nearest').compute()
# draw = phs.IntegrationUtils.DrawMask(waxs_mask_img)

# Load masks:
# draw.load(maskPath.joinpath('WAXS_detector.json'))
# waxs_mask = draw.mask
waxs_mask = np.load(maskPath.joinpath('2024C2_beamstop_only.npy'))

# Check masks:
cmin = float(waxs_mask_img.quantile(0.1))
cmax = float(waxs_mask_img.quantile(0.99999))

sliced_DA = waxs_mask_img.sel(pix_x=pix_x_slice, pix_y=pix_y_slice)
ax = sliced_DA.plot.imshow(norm=LogNorm(cmin,cmax), cmap=cmap, interpolation='antialiased')
ax.axes.imshow(waxs_mask, alpha=0.4, origin='lower')
ax.axes.set(xlim=(300,650), ylim=(300,700), aspect='equal')
ax.figure.set(dpi=150)
# ax.axes.imshow(WAXSinteg.mask, alpha=0.5, origin='lower')
plt.show()
# plt.close('all')

In [ ]:
# Initalize PFEnergySeriesIntegrator object & check beamcenter & masks
# WAXS
# WAXSinteg = phs.integrate.PFGeneralIntegrator(geomethod='template_xr', template_xr=sample_DA.sel(polarization=0))

for energy in loaded_DS.energy.values:
    sample_DA = loaded_DS['raw_intensity'].sel(energy=energy, method='nearest').sel(sample_name='PM7_CF', polarization=90)
    waxs_mask_img = sample_DA.compute()
    WAXSinteg = phs.integrate.PFEnergySeriesIntegrator(geomethod='template_xr', template_xr=sample_DA.drop_vars('energy'))
    WAXSinteg.mask = waxs_mask
    WAXSinteg.ni_beamcenter_x = waxs_mask_img.beamcenter_x
    WAXSinteg.ni_beamcenter_y = waxs_mask_img.beamcenter_y
    print('WAXS Beamcenter: \n'
          f'poni1: {WAXSinteg.poni1}, poni2: {WAXSinteg.poni2} \n'
          f'ni_beamcenter_y: {WAXSinteg.ni_beamcenter_y}, ni_beamcenter_x: {WAXSinteg.ni_beamcenter_x}')

    phs.IntegrationUtils.Check.checkAll(WAXSinteg, 
                                        waxs_mask_img, 
                                        img_scaling='linear', 
                                        img_min=waxs_mask_img.quantile(0.0001), 
                                        img_max=waxs_mask_img.quantile(0.995), 
                                        d_inner=115,
                                        d_outer=310,
                                        alpha=0.4)

    plt.gcf().set(dpi=150)
    plt.show()
    plt.close('all')

In [ ]:
# # Initalize PFEnergySeriesIntegrator object & check beamcenter & masks
# # WAXS
# # WAXSinteg = phs.integrate.PFGeneralIntegrator(geomethod='template_xr', template_xr=sample_DA.sel(polarization=0))

# for dt_val in loaded_DS.datetime.values:
#     sample_DA = loaded_DS['raw_intensity'].sel(datetime=dt_val)
#     waxs_mask_img = sample_DA.compute()
#     WAXSinteg = phs.integrate.PFEnergySeriesIntegrator(geomethod='template_xr', template_xr=sample_DA.drop_vars('energy'))
#     WAXSinteg.mask = waxs_mask
#     WAXSinteg.ni_beamcenter_x = waxs_mask_img.beamcenter_x
#     WAXSinteg.ni_beamcenter_y = waxs_mask_img.beamcenter_y
#     print('WAXS Beamcenter: \n'
#           f'poni1: {WAXSinteg.poni1}, poni2: {WAXSinteg.poni2} \n'
#           f'ni_beamcenter_y: {WAXSinteg.ni_beamcenter_y}, ni_beamcenter_x: {WAXSinteg.ni_beamcenter_x}')

#     phs.IntegrationUtils.Check.checkAll(WAXSinteg, 
#                                         waxs_mask_img, 
#                                         img_scaling='linear', 
#                                         img_min=waxs_mask_img.quantile(0.0001), 
#                                         img_max=waxs_mask_img.quantile(0.995), 
#                                         d_inner=115,
#                                         d_outer=310,
#                                         alpha=0.4)

#     plt.gcf().set(dpi=150)
#     plt.show()
#     plt.close('all')

In [ ]:
# # Tweaking if needed:

# ## WAXS Tweaking & Plot Check
# waxs_new_bcx = 477
# waxs_new_bcy = 508
# WAXSinteg.ni_beamcenter_x = waxs_new_bcx
# WAXSinteg.ni_beamcenter_y = waxs_new_bcy
# WAXSinteg.mask = waxs_mask
# raw_waxs = waxs_mask_img.copy()
# raw_waxs.attrs['beamcenter_x'] = waxs_new_bcx
# raw_waxs.attrs['beamcenter_x'] = waxs_new_bcx
# raw_waxs.attrs['poni1'] = WAXSinteg.poni1
# raw_waxs.attrs['poni2'] = WAXSinteg.poni2

# print('WAXS Beamcenter Tweaking: \n'
#       f'poni1: {WAXSinteg.poni1}, poni2: {WAXSinteg.poni2} \n'
#       f'ni_beamcenter_y: {WAXSinteg.ni_beamcenter_y}, ni_beamcenter_x: {WAXSinteg.ni_beamcenter_x}')
# phs.IntegrationUtils.Check.checkAll(WAXSinteg, 
#                                     waxs_mask_img, 
#                                     img_min=waxs_mask_img.quantile(0.2), 
#                                     img_max=waxs_mask_img.quantile(0.999), 
#                                     d_inner=115,
#                                     d_outer=300,
#                                     alpha=0)
# # plt.xlim(WAXSinteg.ni_beamcenter_x-500, WAXSinteg.ni_beamcenter_x+500)
# # plt.ylim(WAXSinteg.ni_beamcenter_y-500, WAXSinteg.ni_beamcenter_y+500)
# plt.gcf().set(dpi=150)
# plt.show()
# plt.close('all')


# # # Using Pete D.'s (very slightly modified) beamcentering script:
# # phs.BeamCentering.CenteringAccessor.refine_geometry

# # ## WAXS
# # # res_waxs = raw_waxs.sel(pol=0).util.refine_geometry(energy=275, q_min=0.02, q_max=0.06, chi_min=-10, chi_max=70)
# # res_waxs = sample_DA.expand_dims({'energy':[sample_DA.energy]}).util.refine_geometry(energy=1180, q_min=0.28, q_max=0.35)
# # sample_DA.attrs['poni1'] = res_waxs.x[0]
# # sample_DA.attrs['poni2'] = res_waxs.x[1]
# # WAXSinteg = phs.integrate.PFEnergySeriesIntegrator(geomethod='template_xr', template_xr = sample_DA.drop_vars('energy'))
# # WAXSinteg.mask = waxs_mask

# # ## WAXS Plot check
# # print('WAXS Beamcenter Post-optimization: \n'
# #       f'poni1: {WAXSinteg.poni1}, poni2: {WAXSinteg.poni2} \n'
# #       f'ni_beamcenter_y: {WAXSinteg.ni_beamcenter_y}, ni_beamcenter_x: {WAXSinteg.ni_beamcenter_x}')
# # phs.IntegrationUtils.Check.checkAll(WAXSinteg, 
# #                                     waxs_mask_img, 
# #                                     img_scaling='linear', 
# #                                     img_min=waxs_mask_img.quantile(0.01), 
# #                                     img_max=waxs_mask_img.quantile(0.99), 
# #                                     d_inner=215,
# #                                     d_outer=310,
# #                                     alpha=0.4)
# # plt.xlim(WAXSinteg.ni_beamcenter_x-250, WAXSinteg.ni_beamcenter_x+250)
# # plt.ylim(WAXSinteg.ni_beamcenter_y-250, WAXSinteg.ni_beamcenter_y+250)
# # plt.gcf().set(dpi=120)
# # plt.show()
# # plt.close('all')

In [ ]:
# ## WAXS Plot check
# print('WAXS Beamcenter Post-optimization: \n'
#       f'poni1: {WAXSinteg.poni1}, poni2: {WAXSinteg.poni2} \n'
#       f'ni_beamcenter_y: {WAXSinteg.ni_beamcenter_y}, ni_beamcenter_x: {WAXSinteg.ni_beamcenter_x}')
# phs.IntegrationUtils.Check.checkAll(WAXSinteg, 
#                                     waxs_mask_img, 
#                                     img_scaling='linear', 
#                                     img_min=waxs_mask_img.quantile(0.01), 
#                                     img_max=waxs_mask_img.quantile(0.99), 
#                                     d_inner=215,
#                                     d_outer=310,
#                                     alpha=0.4)
# plt.xlim(WAXSinteg.ni_beamcenter_x-500, WAXSinteg.ni_beamcenter_x+500)
# plt.ylim(WAXSinteg.ni_beamcenter_y-500, WAXSinteg.ni_beamcenter_y+500)
# plt.gcf().set(dpi=120)
# plt.show()
# plt.close('all')

In [ ]:
# ### Write beamcenters to saved .json file if content with them:

# beamcenters_dict = {
#     f'WAXS_2023C2': {'bcx':sample_DA.beamcenter_x, 'bcy':sample_DA.beamcenter_y}
# }

# # Check if the file exists, if not, create an empty JSON file
# jsonFile = jsonPath.joinpath('beamcenters_dict.json')
# if not jsonFile.exists():
#     with jsonFile.open('w') as f:
#         json.dump({}, f)

# # Now, read the existing or empty JSON file
# with jsonFile.open('r') as f:
#     dic = json.load(f)

# dic.update(beamcenters_dict)

# # Write the updated dictionary back to the JSON file
# with jsonFile.open('w') as f:
#     json.dump(dic, f)

## Convert to chi-q space & save zarrs

In [ ]:
polar_DA
for dt_val in loaded_DS.datetime.values:
    sample_DA = loaded_DS['raw_intensity'].sel(datetime=dt_val)
    polar_DA = WAXSinteg.integrateImageStack_dask(sample_DA, chunksize=1)
    

In [ ]:
loaded_DS

In [ ]:
# Integrate whole cartesian dataset!
polar_DS_sample_rows = []
for sample_name in tqdm(loaded_DS.sample_name.data[:]):
    polar_DS = xr.Dataset()
    polar_DA_polarization_rows = []
    for pol in [0, 90]:
        cart_DA = loaded_DS['raw_intensity'].sel(polarization=pol, sample_name=sample_name)  #.compute()
        polar_DA = WAXSinteg.integrateImageStack_dask(cart_DA, chunksize=1)
        # polar_DA = WAXSinteg.integrateImageStack(cart_DA)
        # polar_DA = WAXSinteg.integrateSingleImage(cart_DA)
        # polar_DA = WAXSinteg.integrateImageStack_legacy(cart_DA)

        polar_DA = polar_DA.expand_dims({'polarization': [pol]})
        polar_DA_polarization_rows.append(polar_DA)

    polar_DS['raw_intensity'] = xr.concat(polar_DA_polarization_rows, dim='polarization')

    polar_DS = polar_DS.expand_dims({'sample_name':[sample_name]})
    polar_DS_sample_rows.append(polar_DS)
    polar_DS.attrs['name'] = loaded_DS.name
    
    # polar_DS.to_netcdf(zarrsPath.joinpath('polar_rsoxs_carbon_ncs', f'{sample_name}.nc'), format='netCDF4', engine='h5netcdf')
    
polar_DS = xr.concat(polar_DS_sample_rows, dim='sample_name')

In [ ]:
polar_DS

In [ ]:
polar_DS['raw_intensity'].sel(sample_name='PM6-Y6_CF',polarization=90).sel(energy=285,method='nearest').plot.imshow(cmap=cmap)
plt.show()

In [ ]:
def make_para_perp_DAs(DS, sample_name, intensity_type, pol, qlims, chi_width):
    # select dataarray to plot
    DA = DS.sel(sample_name=sample_name)[f'{intensity_type}_intensity']
    sliced_DA = DA.sel(polarization=pol, q=slice(qlims[0],qlims[1]))

    # calculate ISI dataarrays
    if pol==0:
        para_DA = sliced_DA.rsoxs.slice_chi(180, chi_width=(chi_width/2))
        perp_DA = sliced_DA.rsoxs.slice_chi(90, chi_width=(chi_width/2))
    elif pol==90:
        perp_DA = sliced_DA.rsoxs.slice_chi(180, chi_width=(chi_width/2))
        para_DA = sliced_DA.rsoxs.slice_chi(90, chi_width=(chi_width/2))   
        
    return para_DA, perp_DA

In [ ]:
DS

In [ ]:
whos

In [ ]:
polar_DS

In [ ]:
para_DA.sel(energy=285,method='nearest').plot.imshow()
plt.show()

In [ ]:
# Select polar dataset, to qr linecuts: para, perp, and full
edge = 'carbon'
DS = polar_DS.copy()

chi_width = 90
q_slice = slice(0.009,0.08)
e_slice = slice(282, 292)

# make selection
# sample_name = 'PM6-Y6_3000_dSiN'
intensity_type = 'raw'
pol = 0

# for sample_name in tqdm(filtered_selected_samples):
for sample_name in DS.sample_name.values[:1]:
    print('next')
    # for pol in [0]:
    #     para_DA, perp_DA = make_para_perp_DAs(rsoxs_datasets, sample_name, edge, intensity_type, pol, qlims, chi_width) 
   
    pol_paras = []
    pol_perps = []
    for pol in [0, 90]:
        para_DA, perp_DA = make_para_perp_DAs(DS, sample_name, intensity_type, pol, (q_slice.start,q_slice.stop), chi_width) 
        pol_paras.append(para_DA)
        pol_perps.append(perp_DA)


    pol_paras[0] = pol_paras[0].assign_coords({'chi': np.linspace(0, chi_width, len(pol_paras[0].chi.values))})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_paras[1].chi.values))})
    pol_paras[1] = pol_paras[1].assign_coords({'chi': np.linspace(0, chi_width, len(pol_paras[1].chi.values))})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_paras[1].chi.values))})
    pol_perps[0] = pol_perps[0].assign_coords({'chi': np.linspace(0, chi_width, len(pol_perps[0].chi.values))})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_perps[1].chi.values))})
    pol_perps[1] = pol_perps[1].assign_coords({'chi': np.linspace(0, chi_width, len(pol_perps[1].chi.values))})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_perps[1].chi.values))})

    pol_paras[0] = pol_paras[0].interp({'chi': pol_paras[1].chi.values})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_paras[1].chi.values))})
    pol_perps[0] = pol_perps[0].interp({'chi': pol_perps[1].chi.values})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_perps[1].chi.values))})

    para_DA = (pol_paras[0] + pol_paras[1])/2
    perp_DA = (pol_perps[0] + pol_perps[1])/2

    para_DA = para_DA.assign_coords({'polarization':'avg'})
    perp_DA = perp_DA.assign_coords({'polarization':'avg'})

    para_DA = para_DA.interpolate_na(dim='q')
    perp_DA = perp_DA.interpolate_na(dim='q')

    pol = str(para_DA.polarization.values) 
    
#     # Fl correction?
#     scale_factor = float(para_DA.sel(energy=slice(285,305), q=slice(2e-2,6e-2)).mean('chi').mean('energy').integrate('q'))
#     manual_scale_factor = manual_scale_factors_v4[sample_name]
#     trmsn90_corr = (manual_scale_factor * scale_factor * trmsn90_bkgs_DA.sel(sample_name=sample_name))
    
#     para_DA = para_DA - trmsn90_corr
#     perp_DA = perp_DA - trmsn90_corr
                
    # Plot
    fig, axs = plt.subplots(1, 2, figsize=(11,5))
    cmin = 3e8
    cmax = 1e11

    para_slice = para_DA.mean('chi').sel(q=q_slice, energy=e_slice)  # .plot(ax=axs[0], cmap=cmap, norm=LogNorm(cmin, cmax), add_colorbar=False)
    perp_slice = perp_DA.mean('chi').sel(q=q_slice, energy=e_slice)  # .plot(ax=axs[1], cmap=cmap, norm=LogNorm(cmin, cmax), add_colorbar=False)

    # cmin = para_slice.compute().quantile(0.01)
    # cmax = para_slice.compute().quantile(0.995)

    para_slice.plot(ax=axs[0], cmap=cmap, norm=LogNorm(cmin, cmax), add_colorbar=False)
    perp_slice.plot(ax=axs[1], cmap=cmap, norm=LogNorm(cmin, cmax), add_colorbar=False)

    # Add colorbar
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=LogNorm(cmin, cmax)) # Create a ScalarMappable object with the colormap and normalization & add the colorbar to the figure
    cax = axs[1].inset_axes([1.03, 0, 0.05, 1])
    cbar = fig.colorbar(sm, cax=cax, orientation='vertical')
    cbar.set_label(label='Double-Norm-Corrected Intensity [arb. units]', labelpad=12, rotation=270)

    fig.suptitle(f'Linecut Maps: {sample_name}, Polarization = {pol}°, Chi Width = {chi_width}°', fontsize=14)
    # fig.suptitle(f'Linecut Maps: {sample_name} Bare SiN Subtracted, Polarization = {pol}°, Chi Width = {chi_width}°', fontsize=14)
    # fig.suptitle(f'Linecut Maps: {sample_name} Bare SiN & Fluorescence Subtracted, Polarization = {pol}°, Chi Width = {chi_width}°', fontsize=14)

    fig.set(tight_layout=True)

    axs[0].set(xscale='log', title='Parallel to $E_p$', ylabel='Photon energy [eV]', xlabel='q [$Å^{-1}$]')
    axs[1].set(xscale='log', title='Perpendicular to $E_p$ ', ylabel=None, xlabel='q [$Å^{-1}$]')

    # fig.savefig(plotsPath.joinpath('linecut_maps_carbon_v2', f'{sample_name}_{edge}_{energy_min}-{energy_max}_{intensity_type}_chiWidth-{chi_width}deg_pol{pol}deg.png'), dpi=120)

    plt.show()
    plt.close('all')

In [ ]:
for sample_name in loaded_DS.sample_name.values:
# for sample_name in ['PM6_5CN-CB', 'PM6_5CN-CB_rot']:
    DA = loaded_DS['raw_intensity'].sel(sample_name=sample_name, polarization=90).squeeze()
    cmin = float(DA.compute().quantile(0.0001))
    cmax = float(DA.compute().quantile(0.995))
    ax = DA.plot.imshow(norm=plt.Normalize(cmin, cmax), cmap=cmap, x='qx', y='qy')
    ax.axes.set(title=f'{str(DA.sample_name.values)}, Energy = {int(DA.energy.values)} eV')
    plt.show()
    plt.close('all')

In [ ]:
energy = 284.4
# energy = 400
# energy = 530
for sample_name in polar_DS.sample_name.values:
# for sample_name in ['PM6_5CN-CB', 'PM6-Y6_p5CN-2CF-3CB']:
    sliced_DA = polar_DS['corr_intensity'].sel(sample_name=sample_name, polarization=0).sel(energy=energy, method='nearest')
    cmin = sliced_DA.compute().quantile(0.0001)
    cmax = sliced_DA.compute().quantile(0.995)
    ax = sliced_DA.plot.imshow(norm=plt.Normalize(cmin, cmax), cmap=cmap)
    ax.axes.set(title=f'{sample_name}, {int(sliced_DA.energy.values)} eV')
    plt.show()
    plt.close('all')

In [ ]:
energy = 284.4
# energy = 400
# energy = 530
# for sample_name in polar_DS.sample_name.values:
for sample_name in ['PM6_5CN-CB', 'PM6-Y6_p5CN-2CF-3CB']:
    sliced_DA = polar_DS['raw_intensity'].sel(sample_name=sample_name, polarization=0).sel(energy=energy, method='nearest')
    cmin = sliced_DA.compute().quantile(0.0001)
    cmax = sliced_DA.compute().quantile(0.995)
    ax = sliced_DA.plot.imshow(norm=plt.Normalize(cmin, cmax), cmap=cmap)
    ax.axes.set(title=f'{sample_name}, {int(sliced_DA.energy.values)} eV')
    plt.show()
    plt.close('all')

In [ ]:
loaded_polar_DS = xr.open_zarr(zarrsPath.joinpath('polar_rsoxs_carbon_tilted_v1.zarr'))

In [ ]:
loaded_polar_DS

In [ ]:
energy = 285
# energy = 400
# energy = 530
for sample_name in loaded_polar_DS.sample_name.values:
    for pol in [0,45,90]:
        sliced_DA = loaded_polar_DS['corr_intensity'].sel(sample_name=sample_name, polarization=pol).sel(energy=energy, method='nearest')
        cmin = sliced_DA.compute().quantile(0.0001)
        cmax = sliced_DA.compute().quantile(0.995)
        ax = sliced_DA.plot.imshow(norm=plt.Normalize(cmin, cmax), cmap=cmap)
        ax.axes.set(title=f'{sample_name}, pol={pol}°, {int(sliced_DA.energy.values)} eV')
        plt.show()
        plt.close('all')

In [ ]:
def make_para_perp_DAs(DS, sample_name, intensity_type, pol, qlims, chi_width):
    # select dataarray to plot
    DA = DS.sel(sample_name=sample_name)[f'{intensity_type}_intensity']
    sliced_DA = DA.sel(polarization=pol, q=slice(qlims[0],qlims[1]))

    # calculate ISI dataarrays
    if pol==0:
        para_DA = sliced_DA.rsoxs.slice_chi(180, chi_width=(chi_width/2))
        perp_DA = sliced_DA.rsoxs.slice_chi(90, chi_width=(chi_width/2))
    elif pol==90:
        perp_DA = sliced_DA.rsoxs.slice_chi(180, chi_width=(chi_width/2))
        para_DA = sliced_DA.rsoxs.slice_chi(90, chi_width=(chi_width/2))   
        
    return para_DA, perp_DA


# make selection
sample_name = 'PM7_CF'
edge = 'carbon'
intensity_type = 'raw'
pol = 0
qlims = (0.01, 0.08)
chi_width = 30

para_DA, perp_DA = make_para_perp_DAs(polar_DS, sample_name, intensity_type, pol, qlims, chi_width)  

# slice ISI data
para_ISI = para_DA.interpolate_na(dim='q').mean('chi').sum('q')
perp_ISI = perp_DA.interpolate_na(dim='q').mean('chi').sum('q')

# plot
fig, ax = plt.subplots()
para_ISI.sel(energy=slice(280,290)).plot.line(ax=ax, label='para', yscale='log')
perp_ISI.sel(energy=slice(280,290)).plot.line(ax=ax, label='perp', yscale='log')
fig.suptitle('Integrated Scattering Intensity (ISI)', fontsize=14)
ax.set(title=f'{sample_name}, Polarization = {pol}°, Chi Width = {chi_width}°', xlabel='Photon Energy [eV]', ylabel='Double-Norm-Corrected Intensity [arb. units]')
ax.legend()
plt.show()

In [ ]:
# # polar_sample_DS = polar_DS_sample_rows[0]
# for polar_sample_DS in tqdm(polar_DS_sample_rows):
#     # display(polar_sample_DS)
#     sample_name = polar_sample_DS.sample_name.values[0]
#     print(sample_name)
#     polar_sample_DS.to_netcdf(zarrsPath.joinpath('polar_rsoxs_carbon_ncs', f'{sample_name}.nc'), format='netCDF4', engine='h5netcdf')

In [ ]:
polar_DS

In [ ]:
polar_DS

In [ ]:
polar_DS = polar_DS.chunk({'sample_name':1, 'energy':114, 'polarization':1})

# polar_DS['samp_diode'] = DS['samp_diode']
# polar_DS['smoothed_diode'] = DS['smoothed_diode']


polar_DS

In [ ]:
zarrsPath

In [ ]:
[f.name for f in zarrsPath.glob('*')]

In [ ]:
polar_DS

In [ ]:
# # rsoxs_1180
# polar_DS.to_zarr(zarrsPath.joinpath('polar_rsoxs_1180.zarr'), mode='w')

# rsoxs_carbon
polar_DS.to_zarr(zarrsPath.joinpath('polar_rsoxs_carbon_v2.zarr'), mode='w')

# # rsoxs_carbon_reverse
# polar_DS.to_zarr(zarrsPath.joinpath('polar_rsoxs_carbon_reverse_v1.zarr'), mode='w')

# # rsoxs_carbon_tilted
# polar_DS.to_zarr(zarrsPath.joinpath('polar_rsoxs_carbon_tilted_v1.zarr'), mode='w')

In [ ]:
polar_DS.sample_name.values

In [ ]:
sample_names = polar_DS.sample_name.values

polar_DS.sel(sample_name=[sample_names[0]]).to_zarr(zarrsPath.joinpath(f'polar_{polar_DS.name}_raw_v2.zarr'), mode='w')

for sample_name in tqdm(sample_names[1:], desc='Samples...'):
    polar_DS.sel(sample_name=[sample_name]).to_zarr(zarrsPath.joinpath(f'polar_{polar_DS.name}_raw_v2.zarr'), mode='a', append_dim='sample_name')

In [ ]:
netcdf_paths = str(zarrsPath.joinpath('polar_rsoxs_carbon_ncs')) + '/*.nc'
netcdf_paths

In [ ]:
polar_DS = xr.open_mfdataset(netcdf_paths)
polar_DS

In [ ]:
polar_DS

In [ ]:
polar_DS.sample_name.values

In [ ]:
# make selection
sample_name = 'Y6BO_p5CN-CF'

para_DA, perp_DA = make_para_perp_DAs(polar_DS, sample_name, intensity_type, pol, qlims, chi_width)   

# Select AR data
ar_DA = (para_DA.mean('chi') - perp_DA.mean('chi')) / (para_DA.mean('chi') + perp_DA.mean('chi'))

# Plot
ax = ar_DA.sel(energy=slice(282,292)).plot(figsize=(8,5), norm=plt.Normalize(-0.6,0.6))
ax.figure.suptitle('Anisotropy Ratio (AR) Map', fontsize=14, x=0.43)
ax.axes.set(title=f'{sample_name}, Polarization = {pol}°, Chi Width = {chi_width}°', ylabel='Photon Energy [eV]', xlabel='q [$Å^{-1}$]')
ax.colorbar.set_label('AR [arb. units]', rotation=270, labelpad=12)
plt.show()

In [ ]:
# make selection
sample_name = 'Y6BO_p5CN-CF'
edge = 'carbon'
intensity_type = 'corr'
pol = 0
qlims = (0.01, 0.08)
chi_width = 90

para_DA, perp_DA = make_para_perp_DAs(polar_DS, sample_name, intensity_type, pol, qlims, chi_width)   

# Select AR data
ar_DA = (para_DA.mean('chi') - perp_DA.mean('chi')) / (para_DA.mean('chi') + perp_DA.mean('chi'))

# Plot
ax = ar_DA.sel(energy=slice(282,292)).plot(figsize=(8,5), norm=plt.Normalize(-0.6,0.6))
ax.figure.suptitle('Anisotropy Ratio (AR) Map', fontsize=14, x=0.43)
ax.axes.set(title=f'{sample_name}, Polarization = {pol}°, Chi Width = {chi_width}°', ylabel='Photon Energy [eV]', xlabel='q [$Å^{-1}$]')
ax.colorbar.set_label('AR [arb. units]', rotation=270, labelpad=12)
plt.show()

In [ ]:
# display(polar_DS.sample_name.values)
# sample_name = 'PM6-Y6_3000_dSiN'

In [ ]:
energy = 285
# energy = 400
# energy = 530
for sample_name in polar_DS.sample_name.values:
    sliced_DA = polar_DS['raw_intensity'].sel(sample_name=sample_name, polarization=90).sel(energy=energy, method='nearest')
    cmin = sliced_DA.compute().quantile(0.0001)
    cmax = sliced_DA.compute().quantile(0.995)
    sliced_DA.plot.imshow(norm=plt.Normalize(cmin, cmax), cmap=cmap)
    plt.show()
    plt.close('all')

In [ ]:
# 1. Get energy values
energy_values = polar_DS.energy.values
energy_slices = [energy_values[i:i+5] for i in range(0, len(energy_values), 5)]
energy_slices

In [ ]:
polar_DS

In [ ]:
stacked_polar_DS = polar_DS.stack(system=('sample_name', 'polarization')).reset_index('system')
stacked_polar_DS

In [ ]:
# Save the first part of the dataset to initialize the Zarr store
first_system = stacked_polar_DS.isel(system=slice(0, 1))
first_system.to_zarr(zarrsPath.joinpath(f'polar_{polar_DS.name}.zarr'), mode='w')

# Iterate over the rest of the systems and append to the Zarr store
for i in tqdm(range(1, len(stacked_polar_DS.system)), desc='Samples...'):
    subset = stacked_polar_DS.isel(system=slice(i, i+1))
    subset.to_zarr(zarrsPath.joinpath(f'polar_{polar_DS.name}.zarr'), mode='a', append_dim='system')

In [ ]:
sample_names = polar_DS.sample_name.values

polar_DS.sel(sample_name=[sample_names[0]], polarization=[0]).to_zarr(zarrsPath.joinpath(f'polar_{polar_DS.name}.zarr'), mode='w')

for sample_name in tqdm(sample_names[1:], desc='Samples...'):
    # polar_DS.sel(sample_name=[sample_name], polarization=[0]).to_zarr(zarrsPath.joinpath(f'polar_{polar_DS.name}.zarr'), mode='a', append_dim='sample_name')
    polar_DS.sel(sample_name=[sample_name], polarization=[90]).to_zarr(zarrsPath.joinpath(f'polar_{polar_DS.name}.zarr'), mode='a', append_dim='polarization')

In [ ]:
# # import zarr

# # sample_names = polar_DS.sample_name.values
# # energy_values = polar_DS.energy.values
# # energy_slices = [energy_values[i:i+5] for i in range(0, len(energy_values), 5)]

# # root_store = zarr.open_group(zarrsPath.joinpath(f'polar_{polar_DS.name}_regions.zarr').as_posix(), mode='a')

# # for sample in sample_names:
# #     # Make sure there's a group for this sample in the Zarr store
# #     if sample not in root_store.group_keys():
# #         root_store.create_group(sample)
    
# #     subset_by_sample = polar_DS.sel(sample_name=sample)
    
# #     for idx, energy_slice in enumerate(energy_slices):
# #         final_subset = subset_by_sample.sel(energy=energy_slice)
        
# #         # Save to the specific group and energy slice within the Zarr store
# #         final_subset.to_zarr(root_store[sample], mode='a', append_dim='energy', consolidated=True)

# sample_names = polar_DS.sample_name.values
# energy_values = polar_DS.energy.values
# energy_slices = [energy_values[i:i+5] for i in range(0, len(energy_values), 5)]

# main_zarr_path = zarrsPath.joinpath(f'polar_{polar_DS.name}_regions.zarr')

# for sample in tqdm(sample_names, desc='Samples...'):
#     subset_by_sample = polar_DS.sel(sample_name=sample)
    
#     # Define path for the sample within the main Zarr store
#     sample_path = main_zarr_path.joinpath(sample)
    
#     for idx, energy_slice in enumerate(energy_slices):
#         final_subset = subset_by_sample.sel(energy=energy_slice)
        
#         # Save to the specific sample path and energy slice within the Zarr store
#         if idx==0:
#             final_subset.to_zarr(sample_path, mode='w', consolidated=True)
#         else:
#             final_subset.to_zarr(sample_path, mode='a', append_dim='energy', consolidated=True)



In [ ]:
def make_para_perp_DAs(datasets, sample_name, edge, intensity_type, pol, qlims, chi_width):
    # select dataarray to plot
    DS = datasets[f'polar_{edge}']
    DA = DS.sel(sample_name=sample_name)[f'{intensity_type}_intensity']
    sliced_DA = DA.sel(polarization=pol, q=slice(qlims[0],qlims[1]))

    # calculate ISI dataarrays
    if pol==0:
        para_DA = sliced_DA.rsoxs.slice_chi(180, chi_width=(chi_width/2))
        perp_DA = sliced_DA.rsoxs.slice_chi(90, chi_width=(chi_width/2))
    elif pol==90:
        perp_DA = sliced_DA.rsoxs.slice_chi(180, chi_width=(chi_width/2))
        para_DA = sliced_DA.rsoxs.slice_chi(90, chi_width=(chi_width/2))   
        
    return para_DA, perp_DA

# load dictionary of rsoxs datasets
rsoxs_datasets = {}
key = 'polar_regions'
key_start = key.split('_')[0]
key_end = key.split('_')[1]        
zarrPath = list(zarrsPath.glob(f'{key_start}*{key_end}.zarr'))[0]
rsoxs_datasets[key] = xr.open_zarr(zarrPath)

# Compute any dask coordiantes
for coord_name, coord_data in rsoxs_datasets[key].coords.items():
    if isinstance(coord_data.data, da.Array):
        rsoxs_datasets[key].coords[coord_name] = coord_data.compute()
            
rsoxs_datasets[key]

In [ ]:
# make selection
edge = 'regions'
intensity_type = 'corr'
qlims = (0.01, 0.08)
chi_width = 30

for sample_name in tqdm(rsoxs_datasets[f'polar_{edge}'].sample_name.data):
    for pol in [0, 90]:
        ### Select para & perp DataArrays
        para_DA, perp_DA = make_para_perp_DAs(rsoxs_datasets, sample_name, edge, intensity_type, pol, qlims, chi_width)  
        
#         ### ISI:
#         # Slice ISI data
#         para_ISI = para_DA.interpolate_na(dim='q').mean('chi').sum('q')
#         perp_ISI = perp_DA.interpolate_na(dim='q').mean('chi').sum('q')

#         # Plot
#         fig, ax = plt.subplots()
#         para_ISI.sel(energy=slice(280,290)).plot.line(ax=ax, label='para', yscale='log')
#         perp_ISI.sel(energy=slice(280,290)).plot.line(ax=ax, label='perp', yscale='log')
#         fig.suptitle('Integrated Scattering Intensity (ISI)', fontsize=14)
#         ax.set(title=f'{sample_name}, Polarization = {pol}°, Chi Width = {chi_width}°', xlabel='Photon Energy [eV]', ylabel='Double-Norm-Corrected Intensity [arb. units]')
#         ax.legend()
#         fig.savefig(plotsPath.joinpath('isi', f'{sample_name}_{edge}_{intensity_type}_chiWidth-{chi_width}deg_pol{pol}deg.png'), dpi=120)
#         plt.close('all')
        
#         ### Linecut Maps:
#         fig, axs = plt.subplots(1, 2, figsize=(11,5))

#         para_DA.mean('chi').sel(energy=slice(282,290)).plot(ax=axs[0], cmap=cmap, norm=LogNorm(1e9, 1e11), add_colorbar=False)
#         perp_DA.mean('chi').sel(energy=slice(282,290)).plot(ax=axs[1], cmap=cmap, norm=LogNorm(1e9, 1e11), add_colorbar=False)

#         sm = plt.cm.ScalarMappable(cmap=cmap, norm=LogNorm(2e10, 1e12)) # Create a ScalarMappable object with the colormap and normalization & add the colorbar to the figure
#         cax = axs[1].inset_axes([1.03, 0, 0.05, 1])
#         cbar = fig.colorbar(sm, cax=cax, orientation='vertical')
#         cbar.set_label(label='Intensity [arb. units]', labelpad=12)
#         fig.suptitle(f'Linecut Maps: {sample_name}, Polarization = {pol}°, Chi Width = {chi_width}°', fontsize=14)
#         fig.set(tight_layout=True)
#         axs[0].set(title='Parallel to $E_p$', ylabel='Photon energy [eV]', xlabel='q [$Å^{-1}$]')
#         axs[1].set(title='Perpendicular to $E_p$ ', ylabel=None, xlabel='q [$Å^{-1}$]')
#         fig.savefig(plotsPath.joinpath('linecut_maps', f'{sample_name}_{edge}_{intensity_type}_chiWidth-{chi_width}deg_pol{pol}deg.png'), dpi=120)
#         plt.close('all')

        ### AR Maps:
        # Select AR data
        ar_DA = (para_DA.mean('chi') - perp_DA.mean('chi')) / (para_DA.mean('chi') + perp_DA.mean('chi'))

        # Plot
        ax = ar_DA.sel(energy=slice(282,292)).plot(figsize=(8,5), norm=plt.Normalize(-0.6, 0.6))
        ax.figure.suptitle('Anisotropy Ratio (AR) Map', fontsize=14, x=0.43)
        ax.axes.set(title=f'{sample_name}, Polarization = {pol}°, Chi Width = {chi_width}°', ylabel='Photon Energy [eV]', xlabel='q [$Å^{-1}$]')
        ax.colorbar.set_label('AR [arb. units]', rotation=270, labelpad=12)
        # ax.figure.savefig(plotsPath.joinpath('ar_maps', f'{sample_name}_{edge}_{intensity_type}_chiWidth-{chi_width}deg_pol{pol}deg.png'), dpi=120)
        plt.show()
        plt.close('all')